### Edit Snowflake credentials in creds.json

Ensure the python package `textdistance` is installed before beginning.
> pip install textdistance

In [1]:
import json

with open('creds.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
    SF_ACCOUNT = data["account"]
    SF_WH = data["warehouse"]
    SF_DB = data["database"]
    SF_SCHEMA = data["schema"]

In [2]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.functions import call_udf
from snowflake.snowpark.functions import col
from snowflake.snowpark.types import *

In [3]:
import pandas as pd
import textdistance

In [4]:
CONNECTION_PARAMETERS = {
    'url': SF_ACCOUNT,
    'ACCOUNT': SF_ACCOUNT,
    'user': username,
    'password': password,
    'schema': SF_SCHEMA,
    'database': SF_DB,
    'warehouse': SF_WH
}

Try to close the session in case you've been running this previously.

In [ ]:
try:
    session.close()
except:
    pass

First we establish our session with our account credentials and information

In [5]:
session = Session.builder.configs(CONNECTION_PARAMETERS).create()

## Let's load some data into a table

We will use this dataset to determine the best matching room descriptions between Expedia and Booking.com

In [6]:
data = pd.read_csv("room_type.csv")
session.write_pandas(data, 'ROOM_TYPE', auto_create_table=True)
rt = session.table('ROOM_TYPE')
df_pandas = rt.to_pandas()

In [7]:
rt.show()

--------------------------------------------------------------------------------------------------
|"Expedia"                                  |"Booking.com"                                       |
--------------------------------------------------------------------------------------------------
|Deluxe Room, 1 King Bed                    |Deluxe King Room                                    |
|Standard Room, 1 King Bed, Accessible      |Standard King Roll-in Shower Accessible             |
|Grand Corner King Room, 1 King Bed         |Grand Corner King Room                              |
|Suite, 1 King Bed (Parlor)                 |King Parlor Suite                                   |
|High-Floor Premium Room, 1 King Bed        |High-Floor Premium King Room                        |
|Traditional Double Room, 2 Double Beds     |Double Room with Two Double Beds                    |
|Room, 1 King Bed, Accessible               |King Room - Disability Access                       |
|Deluxe Ro

Play around with textdistance

In [8]:
df_pandas["similarity_score"] = df_pandas.loc[:, ["Expedia","Booking.com"]].apply(lambda x: textdistance.hamming.similarity(*x), axis=1)
df_pandas

,Expedia,Booking.com,similarity_score
0,"Deluxe Room, 1 King Bed",Deluxe King Room,7
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible,10
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room,22
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite,0
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room,19
...,...,...,...
201,"Room, 1 King Bed, Accessible, Resort View (Ali...",Alii Tower Resort View With King Bed - Mobilit...,4
202,"Room, 1 King Bed, Accessible, View (Rainbow, B...",Rainbow Tower Ocean View With King Bed - Mobil...,6
203,"Room, 1 King Bed, Ocean View (Alii)",Alii Tower Ocean View With King Bed,0
204,"Room, 1 King Bed, Oceanfront (Rainbow)",Rainbow Tower Ocean Front with King Bed,4


### This Snowflake stage is where our udf and dependencies will be saved

In [9]:
session.sql('CREATE OR REPLACE STAGE text_distance_udf').show()

------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|Stage area TEXT_DISTANCE_UDF successfully created.  |
------------------------------------------------------



### Test of the function first in python before saving to Snowflake

In [10]:
def text_distance(str1, str2):
    score = textdistance.hamming.similarity(str1, str2)
    return score

In [11]:
df_pandas.loc[:, ["Expedia","Booking.com"]].apply(lambda x: text_distance(*x), axis=1)

0       7
1      10
2      22
3       0
4      19
       ..
201     4
202     6
203     0
204     4
205    13
Length: 206, dtype: int64

### We've verified the return, so let's register the code as a User Defined Function in Snowflake
This will automatically import our `textdistance` using the Anaconda channel integration

In [12]:
%%time
@udf(name='text_distance', stage_location="@text_distance_udf", is_permanent=True, replace=True, packages=['textdistance'])
def text_distance(str1: str, str2: str) -> int:
    score = textdistance.hamming.similarity(str1, str2)
    return score

The version of package textdistance in the local environment is 4.5.0, which does not fit the criteria for the requirement textdistance. Your UDF might not work when the package version is different between the server and your local environment


CPU times: user 22.8 ms, sys: 4.91 ms, total: 27.7 ms
Wall time: 3.27 s


### Play around with the UDF in Snowflake using Snowpark DataFrame syntax
Then print the results

In [13]:
rt.select(col("\"Expedia\""), col("Booking.com"), \
          call_udf("text_distance", col("\"Expedia\""), col("Booking.com")).as_("scoring_results")).show()


----------------------------------------------------------------------------------------------------------------------
|"Expedia"                                  |"Booking.com"                                       |"SCORING_RESULTS"  |
----------------------------------------------------------------------------------------------------------------------
|Deluxe Room, 1 King Bed                    |Deluxe King Room                                    |7                  |
|Standard Room, 1 King Bed, Accessible      |Standard King Roll-in Shower Accessible             |10                 |
|Grand Corner King Room, 1 King Bed         |Grand Corner King Room                              |22                 |
|Suite, 1 King Bed (Parlor)                 |King Parlor Suite                                   |0                  |
|High-Floor Premium Room, 1 King Bed        |High-Floor Premium King Room                        |19                 |
|Traditional Double Room, 2 Double Beds     |Dou

## Finally, we can save this DataFrame as a table to Snowflake, if we wish

In [14]:
%%time
rt.select(col("\"Expedia\""), col("Booking.com"), \
          call_udf("text_distance", col("\"Expedia\""), col("Booking.com")).as_("scoring_results")).write.mode('overwrite').saveAsTable('room_type_scoring')


CPU times: user 4.33 ms, sys: 1.93 ms, total: 6.26 ms
Wall time: 1.22 s


In [15]:
session.table("room_type_scoring").show()

----------------------------------------------------------------------------------------------------------------------
|"Expedia"                                  |"Booking.com"                                       |"SCORING_RESULTS"  |
----------------------------------------------------------------------------------------------------------------------
|Deluxe Room, 1 King Bed                    |Deluxe King Room                                    |7                  |
|Standard Room, 1 King Bed, Accessible      |Standard King Roll-in Shower Accessible             |10                 |
|Grand Corner King Room, 1 King Bed         |Grand Corner King Room                              |22                 |
|Suite, 1 King Bed (Parlor)                 |King Parlor Suite                                   |0                  |
|High-Floor Premium Room, 1 King Bed        |High-Floor Premium King Room                        |19                 |
|Traditional Double Room, 2 Double Beds     |Dou